In [21]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, constr
from sqlalchemy import Column, Integer, String, JSON
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base

In [2]:
class User(BaseModel):
    id: int
    name = "Jane Doe"

In [3]:
user = User(id='123')
user_x = User(id=123.45)

In [4]:
assert user.id == 123
assert user_x.id == 123
assert isinstance(user_x.id, int)

In [5]:
assert user.name == "Jane Doe"

In [6]:
assert user.__fields_set__ == {"id"}

In [7]:
assert user.dict() == dict(user) == {"id": 123, "name": "Jane Doe"}

In [8]:
user.id = 321
assert user.id == 321

In [9]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None

In [10]:
class Bar(BaseModel):
    apple = "x"
    banana = "y"

In [11]:
class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

In [12]:
m = Spam(foo={"count": 4}, bars=[{"apple": "x1"}, {"apple": "x2"}])
print(f"{m = }")
print(f"{m.dict() = }")

m = Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])
m.dict() = {'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [13]:
Base = declarative_base()

In [14]:
class CompanyOrm(Base):
    __tablename__ = "companies"
    
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

In [15]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]
    
    class Config:
        orm_mode = True

In [16]:
co_orm = CompanyOrm(
    id=123,
    public_key="foobar",
    name="Testing",
    domains=["example.com", "foobar.com"],
)
print(f"{co_orm = }")

co_orm = <__main__.CompanyOrm object at 0x0000018E983BD820>


In [17]:
co_model = CompanyModel.from_orm(co_orm)
print(f"{co_model = }")

co_model = CompanyModel(id=123, public_key='foobar', name='Testing', domains=['example.com', 'foobar.com'])


In [22]:
class SQLModel(Base):
    __tablename__ = "my_table"
    
    id = Column("id", Integer, primary_key=True)
    metadata_ = Column("metadata", JSON)

In [23]:
class ReserveKeyModel(BaseModel):
    metadata: Dict[str, str] = Field(alias="metadata_")
    
    class Config:
        orm_mode = True

In [24]:
sql_model = SQLModel(metadata_={"key": "value"}, id=1)
pydantic_model = ReserveKeyModel.from_orm(sql_model)
print(f"{pydantic_model = }")
print(f"{pydantic_model.dict(by_alias=True) = }")

pydantic_model = ReserveKeyModel(metadata={'key': 'value'})
pydantic_model.dict(by_alias=True) = {'metadata_': {'key': 'value'}}


In [25]:
class PetClass:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

In [26]:
class PersonClass:
    def __init__(self, *, name: str, age: float = None, pets: List[PetClass]):
        self.name = name
        self.age = age
        self.pets = pets

In [27]:
class Pet(BaseModel):
    name: str
    species: str
    
    class Config:
        orm_mode = True

In [28]:
class Person(BaseModel):
    name: str
    age: float = None
    pets: List[Pet]
    
    class Config:
        orm_mode = True

In [29]:
bones = PetClass(name="Bones", species="dog")
orion = PetClass(name="Orion", species="cat")
anna = PersonClass(name="Anna", age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
print(f"{anna_model = }")

anna_model = Person(name='Anna', age=20.0, pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')])
